In [1]:
import matplotlib.pylab as plt

In [ ]:
plt.imread('demo/1.jpg').shape

In [2]:
import os
import numpy as np
import argparse
import torch
# from src.dataset import CocoDataset
from src.transform import SSDTransformer
import cv2
import shutil

from src.utils import generate_dboxes, Encoder, colors
from src.model import SSD, ResNet

import src
import albumentations as A

/home/sk/SSD-pytorch/src/process.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/home/sk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/sk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/sk/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for

In [3]:
import os
import functools, fnmatch, itertools
import pathlib
def find_files(dir_path: str=None, patterns: [str]=None) -> [str]:
    """
    Returns a generator yielding files matching the given patterns
    :type dir_path: str
    :type patterns: [str]
    :rtype : [str]
    :param dir_path: Directory to search for files/directories under. Defaults to current dir.
    :param patterns: Patterns of files to search for. Defaults to ["*"]. Example: ["*.json", "*.xml"]
    """
    path = dir_path or "."
    path_patterns = patterns or ["*"]
    fn = []
    for root_dir, dir_names, file_names in os.walk(path):

        filter_partial = functools.partial(fnmatch.filter, file_names)

        for file_name in itertools.chain(*map(filter_partial, path_patterns)):
             fn.append(pathlib.Path(os.path.join(root_dir, file_name)))

    return sorted(fn)

In [4]:
model = SSD(backbone=ResNet(), num_classes=2)

In [5]:
checkpoint = torch.load('trained_models/Invivo_heavy_aug_train/best_SSD_Invivo_heavy_aug_train.pth')
model.load_state_dict(checkpoint["model_state_dict"])

if torch.cuda.is_available():
    model.cuda()
model.eval()
dboxes = generate_dboxes(model="ssd")
encoder = Encoder(dboxes)

In [ ]:
# img_path = '/home/sk/Rewire_Image/Rewire_original_models/c-fos/test/'

# img_path_list = find_files(img_path, ['*.jpg'])
# target_path_list = find_files(img_path, ['*.json'])


In [6]:

Invivo_val_transform = A.Compose([
    A.LongestMaxSize(300),
    A.PadIfNeeded(300,300, border_mode=0)],
    bbox_params=A.BboxParams(format='coco', min_area=15, min_visibility=0.1, label_fields=['class_labels']))


In [7]:
train_path = '/home/sk/repo/FishCam/splitted_anno_0/'

In [13]:
train_set = src.coco_dataset.CocoTrainDataset(train_path, 'val', dboxes, img_aug = Invivo_val_transform)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [10]:
# self.train_set = src.coco_dataset.CocoTrainDataset(self.train_path, 'train', self.dboxes, img_aug = self.train_img_aug)
# self.train_loader = DataLoader(self.train_set, batch_size = self.batch_size, shuffle = True, num_workers = self.num_workers)

In [14]:
output_path = 'Invivo_heavy_aug_test'

In [15]:
if os.path.isdir(output_path):
    shutil.rmtree(output_path)
os.makedirs(output_path)

In [16]:
nms_threshold = 0.2
cls_threshold = 0.001

In [17]:
for i, (img,  _, _) in enumerate(train_set):
    if img is None:
        continue
    if torch.cuda.is_available():
        img = img.cuda()
    with torch.no_grad():
        ploc, plabel = model(img.unsqueeze(dim=0))
        result = encoder.decode_batch(ploc, plabel, nms_threshold, max_output= 30)[0]
        loc, label, prob = [r.cpu().numpy() for r in result]
        best = np.argwhere(prob > cls_threshold).squeeze(axis=1)
        loc = loc[best]
        label = label[best]
        prob = prob[best]
        # print(len(loc))
        if len(loc) > 0:
            np_img = img.permute(1, 2, 0).cpu().numpy()
            opencv_img = cv2.cvtColor(cv2.normalize(np_img, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8U), cv2.COLOR_RGB2BGR)
            # path = test_set.coco.loadImgs(img_id)[0]["file_name"]
            height, width= np_img.shape[:2]
            loc[:, 0::2] *= width
            loc[:, 1::2] *= height
            loc = loc.astype(np.int32)
            
            for box, lb, pr in zip(loc, label, prob):
                category = 'Fish'
                color = colors[lb]
                xmin, ymin, xmax, ymax = box
                opencv_img= cv2.rectangle(opencv_img, (xmin, ymin), (xmax, ymax), color, 1)
                # text_size = cv2.getTextSize(category + " : %.2f" % pr, cv2.FONT_HERSHEY_PLAIN, 0.01, 1)[0]
                # cv2.rectangle(opencv_img, (xmin, ymin), (xmin + text_size[0] + 3, ymin + text_size[1] + 4), color, 1)
                # cv2.putText(
                #     opencv_img, category + " : %.2f" % pr,
                #     (xmin, ymin + text_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1,
                #     (255, 255, 255), 1)
            cv2.imwrite("{}/{}_prediction.jpg".format(output_path, i), opencv_img)


In [ ]:
category

In [ ]:
plt.imshow(opencv_img)

In [ ]:
import torch.nn.functional as F
from torchvision.ops.boxes import box_iou, box_convert


In [ ]:
ploc, plabel = model(img.unsqueeze(dim=0))
bboxes, probs = encoder.scale_back_batch(ploc, plabel)
output = []
for bbox, prob in zip(bboxes.split(1, 0), probs.split(1, 0)):
    bbox = bbox.squeeze(0)
    prob = prob.squeeze(0)
    print(decode_single(bbox, prob, 0.5, max_output=100))
    break

In [ ]:
bbox[1, :][mask, :]

In [ ]:
bboxes_out = []
scores_out = []
labels_out = []

for i, score in enumerate(scores_in):
    # if i == 0:
    #     continue
    # print(score)
    # score = score.squeeze(1)
    # mask = score > 0.003
    # print(mask)
    # # bboxes, score = bboxes_in[mask, :], score[mask]
    # bboxes, score = bbox[mask, :], score[mask]
    # if score.size(0) == 0: continue

    # score_sorted, score_idx_sorted = score.sort(dim=0)

    # # select max_output indices
    # score_idx_sorted = score_idx_sorted[-max_num:]
    # candidates = []

    # while score_idx_sorted.numel() > 0:
    #     idx = score_idx_sorted[-1].item()
    #     bboxes_sorted = bboxes[score_idx_sorted, :]
    #     bboxes_idx = bboxes[idx, :].unsqueeze(dim=0)
    #     iou_sorted = box_iou(bboxes_sorted, bboxes_idx).squeeze()
    #     # we only need iou < nms_threshold
    #     score_idx_sorted = score_idx_sorted[iou_sorted < nms_threshold]
    #     candidates.append(idx)

    # bboxes_out.append(bboxes[candidates, :])
    # scores_out.append(score[candidates])
    # labels_out.extend([i] * len(candidates))

In [ ]:
for i, score in enumerate(prob):
    print(score)
    mask = score > 0.003
    bboxes_, score_ = bbox[mask, :], score[mask]

In [ ]:
score_.size(0)

In [ ]:
score_.sort(dim=0)

In [ ]:
mask

In [ ]:
score.shape

In [ ]:
prob.shape

In [ ]:
for i, score in enumerate(prob.split(1, 1)):
    if i == 0:
        continue
    print(score)
    break

In [ ]:
mask = score.squeeze(1) > 0.05


In [ ]:
bbox.shape

In [ ]:
mask

In [ ]:
bbox.shape

In [ ]:
bboxes.shape

In [ ]:
mask = prob.squeeze(1) > 0.003

In [ ]:
mask.shape

In [ ]:
torch.sum(mask[0,:])

In [ ]:
mask[0,:]

In [ ]:
bbox[mask[0,:], :]

In [ ]:
for a,b in zip(bboxes, probs):
    print(a.shape,b.shape)
    break

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
plabel

In [ ]:
mask = prob > 0.0001

In [ ]:
mask.shape

In [ ]:
bboxes.shape

In [ ]:
bboxes[mask, :]

In [ ]:
for i, score in enumerate(prob.split(1,1)):
    print(score.squeeze(1) )

In [ ]:
def scale_back_batch(self, bboxes_in, scores_in):
    """
        Do scale and transform from xywh to ltrb
        suppose input Nx4xnum_bbox Nxlabel_numxnum_bbox
    """
    
    self.dboxes = self.dboxes.to(self.device)
    self.dboxes_xywh = self.dboxes_xywh.to(self.device)

    bboxes_in = bboxes_in.permute(0, 2, 1)
    # scores_in = scores_in.permute(0, 2, 1)

    bboxes_in[:, :, :2] = self.scale_xy * bboxes_in[:, :, :2]
    bboxes_in[:, :, 2:] = self.scale_wh * bboxes_in[:, :, 2:]

    bboxes_in[:, :, :2] = bboxes_in[:, :, :2] * self.dboxes_xywh[:, :, 2:] + self.dboxes_xywh[:, :, :2]
    bboxes_in[:, :, 2:] = bboxes_in[:, :, 2:].exp() * self.dboxes_xywh[:, :, 2:]
    bboxes_in = box_convert(bboxes_in, in_fmt="cxcywh", out_fmt="xyxy")

    return bboxes_in, F.softmax(scores_in, dim=-1)

In [ ]:
probs

In [ ]:

t_p = F.softmax(plabel, dim=-1) > 0.001

In [ ]:
t_p.sum()

In [ ]:
encoder.scale_wh

In [ ]:
encoder.dboxes

In [ ]:
encoder.dboxes_xywh

In [ ]:
bbox

In [ ]:
plabel

In [ ]:
prob

In [ ]:
prob.shape

In [ ]:
encoder.decode_single(ploc.permute(0, 2, 1)[0, 0, :], plabel.permute(0, 2, 1)[0, 0, :], 0.05, max_output=100)

In [ ]:
encoder.decode_batch(ploc, plabel, 100, max_output= 10000)[0]

In [ ]:
for r in result:
    print(r)

In [ ]:
ploc.shape

In [ ]:
ploc.permute(0,2,1).shape

In [ ]:
bboxes_in.permute(0, 2, 1)

In [ ]:
plabel

In [ ]:
import matplotlib.pylab as plt

In [ ]:
img.permute(1, 2, 0).cpu().numpy().shape[:2]

In [ ]:
(plabel > 0.1).sum()

In [ ]:

# test_set = CocoDataset(opt.data_path, 2017, "val", SSDTransformer(dboxes, (300, 300), val=True))


# if os.path.isdir(opt.output):
#     shutil.rmtree(opt.output)
# os.makedirs(opt.output)

for img, img_id, img_size, _, _ in test_set:
    if img is None:
        continue
    if torch.cuda.is_available():
        img = img.cuda()
    with torch.no_grad():
        ploc, plabel = model(img.unsqueeze(dim=0))
        result = encoder.decode_batch(ploc, plabel, opt.nms_threshold, 20)[0]
        loc, label, prob = [r.cpu().numpy() for r in result]
        best = np.argwhere(prob > opt.cls_threshold).squeeze(axis=1)
        loc = loc[best]
        label = label[best]
        prob = prob[best]
        if len(loc) > 0:
            path = test_set.coco.loadImgs(img_id)[0]["file_name"]
            output_img = cv2.imread(os.path.join(opt.data_path, "val2017", path))
            height, width, _ = output_img.shape
            loc[:, 0::2] *= width
            loc[:, 1::2] *= height
            loc = loc.astype(np.int32)
            for box, lb, pr in zip(loc, label, prob):
                category = test_set.label_info[lb]
                color = colors[lb]
                xmin, ymin, xmax, ymax = box
                cv2.rectangle(output_img, (xmin, ymin), (xmax, ymax), color, 2)
                text_size = cv2.getTextSize(category + " : %.2f" % pr, cv2.FONT_HERSHEY_PLAIN, 1, 1)[0]
                cv2.rectangle(output_img, (xmin, ymin), (xmin + text_size[0] + 3, ymin + text_size[1] + 4), color,
                              -1)
                cv2.putText(
                    output_img, category + " : %.2f" % pr,
                    (xmin, ymin + text_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1,
                    (255, 255, 255), 1)
                cv2.imwrite("{}/{}_prediction.jpg".format(opt.output, path[:-4]), output_img)


In [ ]:

def get_args():
    parser = argparse.ArgumentParser("Implementation of SSD")
    parser.add_argument("--data-path", type=str, default="/coco", help="the root folder of dataset")
    parser.add_argument("--cls-threshold", type=float, default=0.5)
    parser.add_argument("--nms-threshold", type=float, default=0.5)
    parser.add_argument("--pretrained-model", type=str, default="trained_models/SSD.pth")
    parser.add_argument("--output", type=str, default="predictions")
    args = parser.parse_args()
    return args


def test(opt):
